In [ ]:
from from_root import from_root
from src.snp import make_snp_weeklies

In [ ]:
from ib_insync import *
util.startLoop()

In [ ]:
ROOT = from_root()
file_path = ROOT / 'data' / 'master' / 'snp_indexes.yml'

df = make_snp_weeklies(file_path)

In [ ]:
# Build underlying contracts
contracts = [Stock(symbol=symbol, exchange='SMART', currency='USD') 
             if 
                secType == 'STK' 
             else 
                Index(symbol=symbol, exchange='CBOE', currency='USD') 
             for 
                symbol, secType in zip(df.symbol, df.secType)]

In [ ]:
ib.qualifyContracts(*contracts)

In [ ]:
ib.reqContractDetails(Index('SPXW', currency='USD'))